In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import interp1d
import skimage
from skimage.morphology import skeletonize
from skimage.util import invert

### First step is to load in the image that I want to load in and plot it. 

The purpose of this step is to see where I should crop the image so that we just have an image from the head to the point of departure of the snake. This cropping step is not easily automatable right now and I am using this as a stop gap measure until I find a better method.

In [ ]:
img_path = "snake_filtered/im11.jpg"  # specify the image we want to use.


def plot_img(img: np.ndarray, size: int = 6) -> None:
    """
    Plot an image.

    Parameters
    ----------
    img : np.ndarray
        Image array.
    figsize : int
        Size of figure. Default 8x8.

    Returns
    -------
    """
    plt.figure(figsize=(size, size))
    skimage.io.imshow(img, cmap="gray")
    plt.show()


def load_and_plot(img_path: str) -> np.ndarray:
    """
    Load the image and plot it. I'm plotting the inverse so that the snake has
    high pixel values and the background has low pixel values.

    Parameters
    ----------
    img_path : str
        Path to image.

    Returns
    -------
    img : np.ndarray
        Inverted image array.
    """
    img = skimage.io.imread(img_path)
    img = invert(img)
    img = make_truly_bw(img)
    plot_img(img)
    return img


def make_truly_bw(img):
    """
    Force all pixels to be black or white.

    Parameters
    ----------
    img : np.ndarray
        Grayscale image array.

    Returns
    -------
    img : np.ndarray
        Image containing only black and white pixel values.
    """
    img = 255 * (img > 100)
    return img

In [ ]:
img = load_and_plot(img_path)

### The next step is to crop the image so that the picture only includes the snake from the head to the point of departure

The reason for this is that we are modeling the snake from the tip to the point it leaves the ground and the extra information is superfluous.

In [ ]:
def crop(
    img: np.ndarray, left: int, right: int, top: int, bottom: int = 3300, size: int = 8
) -> np.ndarray:
    """
    Crop the image so only to tip to the point of departure are visible.

    Crop the image horizontally and vertically. Plot the result next to the
    original. The intention is that this function provides some interactivty
    in the cropping process.

    Parameters
    ----------
    img : np.ndarray
        Image array.
    left : int
        Drop all pixels before this on the left of the image.
    right : int
        Drop all pixels after this on the right of the image.
    top : int
        Drop all pixels above this at the top of the image.
    bottom : int
        Drop all pixels below this at the bottom of the image. Default=3300.
        I can provide a default here because we know where the bottom of the
        post is.
    size : int
        Size of plot.

    Returns
    -------
    img : np.ndarray
        Cropped image.
    """
    cropped = img[top:bottom, left:right]
    # plot_img(cropped)
    fig, ax = plt.subplots(1, 2, figsize=(size, size))

    ax[0].imshow(img, cmap=plt.cm.gray)
    ax[1].imshow(cropped, cmap=plt.cm.gray)
    plt.show()
    return cropped

# We must crop the image by hand

In [ ]:
cropped = crop(img, left=325, right=1500, top=1050)

### Now that we have the cropped image we need to do the extraction of the shape.

deFor now I will extract the front of the snake but I will also look into extracting the back and the center line. In reality I think the center line is most appropriate so I will work towards that.

In [ ]:
def extract_skeleton(cropped: np.ndarray) -> np.ndarray:
    """
    Extract the skeleton and return the pixel values describing the shape.

    Parameters
    ----------
    cropped : np.ndarray
        A cropped image of the snake.

    skeleton : np.ndarray
        A 2D array holding the x, y pixel values of the skeleton.
    """
    skeleton_image = skeletonize(cropped / 255, method="lee")
    skeleton = np.argwhere(skeleton_image)
    return skeleton


def extract_front(cropped: np.ndarray) -> np.ndarray:
    """
    Extract the front of the snake from the image and return the pixel values
    describing the extracted shape.

    Parameters
    ----------
    cropped : np.ndarray
        A cropped image of the snake.

    Returns
    -------
    front : np.ndarray
        A 2D array holding the x,y pixel values of the front of the snake.
    """
    rows = []
    columns = []
    for i in range(cropped.shape[0]):
        if np.any(cropped[i]):
            row_idx = i
            rows.append(row_idx)
            column_idx = np.where(cropped[i])[0][0]
            columns.append(column_idx)
    rows = np.array(rows)
    columns = np.array(columns)
    front = np.array(list(zip(columns, rows)))
    return front


def extract_back(cropped: np.ndarray) -> np.ndarray:
    """
    Exctract a line describing the back of the snake.

    Extract the back of the snake from the image and return the pixel values
    describing the extracted shape.

    Parameters
    ----------
    cropped : np.ndarray
        A cropped image of the snake.

    Returns
    -------
    back : np.ndarray
        A 2D array holding the x, y pixel values of the back of the snake.
    """
    rows = []
    columns = []
    for i in range(cropped.shape[1]):
        if np.any(cropped[:, i]):
            col_idx = i
            columns.append(col_idx)
            row_idx = np.where(cropped[:, i])[0][0]
            rows.append(row_idx)
    rows = np.array(rows)
    columns = np.array(columns)
    back = np.array(list(zip(columns, rows)))
    return back


def plot_outline(cropped: np.ndarray, outline: np.ndarray, size: int = 6) -> None:
    """
    Plot an outline of the shape of the snake.

    Parameters
    ----------
    cropped : np.ndarray
        Cropped image of the snake.
    outline : np.ndarray
        A 2D array giving pixel values for an outline.

    Returns
    -------
    """
    plt.figure(figsize=(size, size))
    plt.imshow(cropped, cmap="gray")
    plt.scatter(outline[:, 0], outline[:, 1], s=2, color="red")
    plt.show()

In [ ]:
front = extract_front(cropped)

### Sometimes we may have some erroneous points in the outline from imperfect image editing. Below we have some code to remove those points

For example the outline might have a random dot off the snake like in the image below 

<p align="center">
    <img src="unwanted_point_example.png" alt="unwanted point" width="400"/>
<p/>

It's obvious what point we want to remove from the outline. That red dot floating somewhere around (250, 300). Below is some code to remove the dot. I will make it a function once I have some more practice

In [ ]:
plot_outline(cropped, front)

In [ ]:
# first get the rows and columns
row = front[:, 1]
col = front[:, 0]

# We must remove outliers by hand for now as well

In [ ]:
# now select the rows i.e. where the outlides are. Note this code extracts the
# rows in the arrays where the outliers might be. From the overlap of the two
# arrays we can find the outliers.
rows_of_interest = np.argwhere((row > 200) & (row < 1800))
cols_of_interest = np.argwhere((col < 350))

In [ ]:
def arr_to_set(arr):
    """Convert an array to a set."""
    arr = arr.ravel()
    set_ = set(arr)
    return set_


# convert the arrays to sets
rows_of_interest = arr_to_set(rows_of_interest)
cols_of_interest = arr_to_set(cols_of_interest)

# look at the interesection of the sets. This tells me what row the outlier is
# in.
rows_to_delete = list(rows_of_interest.intersection(cols_of_interest))

In [ ]:
# now delete the row of the image array where the outlier is
front = np.delete(front, rows_to_delete, axis=0)

In [ ]:
plot_outline(cropped, front)

Now it's much better. 

### Now that we have extracted an outline from the image we need to interpolate the outline so we can do curvature analysis.

In order to fit a spline we need to create a parameter to fit on. We will use the distance as our parameter.

In [ ]:
def get_distance_parameter(outline, px_per_cm=36.4):
    """
    Use the outline to construct a distance parameter.

    Parameters
    ----------
    outline : np.ndarray
        Outline shape (front, back, or center) of the snake.
    px_per_cm : float
        Magic number which set the distance scale (in cm) in the image. This
        number comes from knowing a physical distance in the image and seeing
        how many pixels that distance takes up.

    Returns
    ------
    distance : np.ndarray
        This array contains the distance between the outline points (in cm).
    """
    distance = np.cumsum(np.sqrt(np.sum(np.diff(outline, axis=0) ** 2, axis=1)))
    distance = np.insert(distance, 0, 0)
    distance = distance / px_per_cm
    return distance


def get_interp_points(distance, num_points: int = 10):
    """Get the points I will do the interpolation on."""
    interp_points = np.linspace(distance.min(), distance.max(), num_points)
    return interp_points


def get_interpolation(distance: np.ndarray, outline: np.ndarray, num_points: int = 10):
    """
    Get the interpolation of the line.
    """
    interpolator = interp1d(distance, outline, axis=0)
    interp_points = get_interp_points(distance, num_points)
    interpolated_line = interpolator(interp_points)
    return interpolated_line, interpolator


def plot_interpolated_line(
    cropped: np.ndarray,
    outline: np.ndarray,
    interpolated_line: np.ndarray,
    size: int = 8,
):
    """
    Plot the interpolation line over top of the snake image.

    Paramters
    ---------
    cropped : np.ndarray
        Cropped image of the snake.
    outline : np.ndarray
        A 2D array with the outline (front, back, or center) outline of the
        snake. The columns correspond to x and y pixel values.
    interpolated_line : np.ndarray
        A 2D array with the interpolation x and y pixel values for the outline.

    Returns
    -------
    """
    plt.figure(figsize=(size, size))
    plt.imshow(cropped, cmap="gray")
    plt.scatter(outline[:, 0], outline[:, 1], s=50, color="red")
    plt.plot(interpolated_line[:, 0], interpolated_line[:, 1], linewidth=5)
    plt.title("Overlay of interpolated line")
    plt.show()

In [ ]:
distance = get_distance_parameter(outline=front)
interpolated_line, interpolator = get_interpolation(distance, front)

In [ ]:
plot_interpolated_line(cropped, front, interpolated_line)

### Now that we have an outline of the snake extracted and we have a fit to the outline, we want to extract the curvature profile and plot that. 

We don't have an arc length parametrization because the magnitude of the tangent vector is not one. Instead we have a proper parameterization because the derivatives $\frac{dx}{dt}$ and $\frac{dy}{dt}$ exist everywhere and are not zero simultatneously. To compute curvature we can then use the [following formula](https://en.wikipedia.org/wiki/Curvature)

\begin{equation}
|\kappa| = \frac{|x'y''-x''y'|}{(x'^2+y'^2)^{\frac{3}{2}}}
\end{equation}

In [ ]:
def check_parameterization(
    interpolated_line: np.ndarray, interpolation_points: np.ndarray
) -> bool:
    """
    Check that the parameterization is "proper."

    A proper parameterization means the derivatives dx/dt and dy/dt exist
    everywhere and are not zero simultaneously.

    Parameters
    ----------
    interpolated_line : np.ndarray
        A 2D array containing the x, y pixel values of the interpolation line.
    interpolation_points : np.ndarray
        Points at which the interpolation took place. These are values of the
        parameter we interpolate from.

    Returns
    -------
    proper : bool
        True if the parameterization is "proper" as described above. False otherwise.
    """
    x = interpolated_line[:, 0]
    y = interpolated_line[:, 1]
    t = interpolation_points
    spacing = np.diff(t)[0]
    dx_dt = np.gradient(x, spacing)
    dy_dt = np.gradient(y, spacing)
    deriv_mag = np.sqrt(dx_dt**2 + dy_dt**2)
    no_nans_x = np.all(~np.isnan(dx_dt))
    no_nans_y = np.all(~np.isnan(dy_dt))
    no_nans = no_nans_x and no_nans_y
    non_zero = np.all(deriv_mag > 0)
    proper = no_nans and non_zero
    return proper


def compute_curvature(
    interpolated_line: np.ndarray, interpolation_points: np.ndarray
) -> np.ndarray:
    """
    Compute and plot the curvature.

    Parameters
    ----------
    interpolated_line : np.ndarray
        A 2D array containing the x, y pixel values of the interpolation.
    interpolation_points : np.ndarray
        Array containing the points we will do the interpolation at.

    Returns
    -------
    kappa : np.ndarray
        Array containing the curvature at each point of the interpolation.
    """
    x = interpolated_line[:, 0]
    y = interpolated_line[:, 1]
    t = interpolation_points
    spacing = np.diff(t)[0]
    dx_dt = np.gradient(x, spacing)
    dy_dt = np.gradient(y, spacing)
    d2x_dt = np.gradient(dx_dt, spacing)
    d2y_dt = np.gradient(dy_dt, spacing)
    kappa = np.abs(dx_dt * d2y_dt - d2x_dt * dy_dt) / (
        np.power(dx_dt**2 + dy_dt**2, 3 / 2)
    )
    plt.plot(interpolation_points, kappa)
    plt.xlabel("Distance along snake (cm)")
    plt.ylabel(r"Curvature $\kappa$")
    plt.title("Curvature along the snake.")
    plt.show()
    return kappa

In [ ]:
interpolation_points = get_interp_points(distance=distance)
proper = check_parameterization(
    interpolated_line=interpolated_line, interpolation_points=interpolation_points
)
print(f"The parametrization is proper: {proper}")

In [ ]:
kappa = compute_curvature(interpolated_line, interpolation_points)
plot_interpolated_line(
    cropped=cropped, outline=front, interpolated_line=interpolated_line, size=6
)

## testing out filtered data

In [ ]:
from scipy.signal import savgol_filter

In [ ]:
row = front[:, 1]
col = front[:, 0]
row_filtered = savgol_filter(row, window_length=10, polyorder=3)
col_filtered = savgol_filter(col, window_length=10, polyorder=3)

In [ ]:
np.hstack((row_filtered, col_filtered)).shape

In [ ]:
plt.scatter(row, col)
plt.scatter(row_filtered, col_filtered)
plt.show()